__Cuaderno de trabajo de:__ Nombre Apellido

# Regresión Lineal Múltiple

Vamos a continuar con el capítulo 3 del libro ["Introduction to Statistical Learning"](http://www-bcf.usc.edu/~gareth/ISL/), ahora con la sección 3.2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import seaborn as sns

from sklearn.preprocessing import scale
import sklearn.linear_model as skl_lm
import sklearn.metrics
import statsmodels.api as sm
import statsmodels.formula.api as smf

pd.set_option('display.notebook_repr_html', False)
 
%matplotlib inline
plt.style.use('seaborn-white')

ModuleNotFoundError: No module named 'statsmodels'

## Cargamos los datos

Cargamos los conjuntos de datos que vamos a usar

Los conjuntos de datos están en la web del libro (pero ya están descargados)
http://www-bcf.usc.edu/~gareth/ISL/data.html

In [ ]:
advertising = pd.read_csv('advertising.csv', usecols=[1,2,3,4], na_values='?').dropna()  
# na_values='?').dropna()  borra las filas y ciolumnas que tienen Null/NaN 
advertising.info()

## Regresión y ajuste de modelos

El análisis de regresión consiste en encontrar un  **modelo** que relaciona los valores medidos de una variable **objetivo** (tb se llama la **respuesta**) en función de un conjunto de variables **explicativas** (tb **variables predictoras**, o **variables explicativas**, o **regresores**).

Los valores medidos en el mundo real nunca se ajustan de forma perfecta a un modelo, debido en primer lugar a errores de medida, pero también a que cualquier modelo matemático es una *simplificación* del mundo real, y si tuviera en cuenta todos los factores que influyen en un conjunto de variables, sería inmanejable.

Por tanto, no tiene sentido aspirar a encontrar un modelo que prediga exactamente los valores medidos, y debemos admitir que el modelo cometerá un cierto error.

Un modelo útil encuentra una relación funcional sencilla en conjuntos de pocas variables. Se trata de explicar una variable objetivo en función de otro conjunto de variables mejor conocidas o más fáciles de medir. El  **análisis de regresión**  (más exactamente, el análisis de regresión  *paramétrico*) permite encontrar un modelo explicativo en dos etapas:


 1. Nuestro conocimiento del tema en cuestión nos permite escribir un modelo que afirma que la variable  *Y*  es una función de las variables $X_1,\dots,X_p$. La variable  *Y* se suele llamar la **respuesta** y las variables  $X_1,\dots,X_p$ se llaman  **variables predictoras**. La forma exacta de la función no está fijada a priori, sino que depende de unos pocos  **parámetros**  libres.
 
 Por ejemplo, para la **regresión lineal**, el modelo es
 $$
 Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_p X_p + \epsilon
 $$
 donde $\beta_0,\dots,\beta_p$ son los parámetros y $\epsilon$ es un error que no podemos explicar dentro de este modelo.
 
 2. **Ajustamos el modelo** a los datos de que disponemos, eligiendo los valores de los parámetros para los que la distancia entre los valores medidos de la variable  *Y*  y los valores predichos aplicando el modelo minimizan el error cometido. El error que se suele minimizar es el error cuadrático (**residual sum of squares**):
$$
RSS = e_1^2 + e_2^2 + \dots + e_n^2
$$
donde
$$
e_1 = y_1 - (\beta_0 + \beta_1 X_1^1 + ... + \beta_p X_p^1), \dots e_n = y_n - (\beta_0 + \beta_1 X_1^n  + ... + \beta_p X_p^n).
$$
Es decir, entre todas las posibles combinaciones de coeficientes $(\beta_0,\dots,\beta_p)$ nos quedamos con aquella combinación $(\hat{\beta_0},\hat{\beta_1},\dots,\hat{\beta_1})$ que minimiza el RSS, y hemos obtenido la  **regresión lineal por mínimos cuadrados** (**least squares**).

## Ejemplo: Inversión en publicidad

Hacer regresión múltiple consiste en encontrar una función lineal de las variables predictoras que aproxima la función objetivo:

$$
Sales\approx \beta_0 + \beta_1\times TV  + \beta_2\times radio  + \beta_3\times newspaper
$$

Si incorporamos el término de error, podemos sustituir el signo de aproximación $\approx$ por uno de igualdad:

$$
Sales = \beta_0 + \beta_1\times TV  + \beta_2\times radio  + \beta_3\times newspaper + \varepsilon
$$


que minimiza el error cuadrático. Es cualitiva y cuantitivamente distinto de ajustar modelos por separado para cada variable predictora:

$$
Sales = \beta_0^{TV} + \beta_1^{TV}\times TV + \varepsilon
$$

$$
Sales = \beta_0^R + \beta_1^R\times radio  + \varepsilon
$$

$$
Sales = \beta_0^N + \beta_1^{N}\times newspaper + \varepsilon
$$

### Regresión lineal múltiple con scikit_learn

  1. Definimos un objeto de tipo "LinearRegression"

```python
regr = skl_lm.LinearRegression()
```

  2. El método ``fit`` **"ajusta"** la función lineal, encontrando los valores de $(\beta_0, \beta_1, \dots, \beta_p)$ para los que el error cuadrático cometido es menor. Necesita dos argumentos:
      - Un **array 2D de numpy** o un **DataFrame** X con las variables predictoras. No acepta una serie ni un array 1D.
      - Una **Serie** y con la variable objetivo.

```python
X = advertising[['TV', 'Radio', 'Newspaper']]
y = advertising.Sales

regr.fit(X,y)
```

y ya tenemos el objeto ``regr`` que contiene la recta ajustada por mínimos cuadrados.

In [ ]:
regr = skl_lm.LinearRegression()

X = advertising[['TV', 'Radio', 'Newspaper']]
y = advertising.Sales

regr.fit(X,y)

print(regr.intercept_)  #beta0
print(regr.coef_)       #beta1 .. betap

#### predecir 

Ahora que el modelo está ajustado, podemos predecir la cantidad de ventas, conocido el valor de las variables predictoras.

```python
regr.predict([[inv_TV, inv_Radio, inv_Newspaper]])
```

In [ ]:
2.938889369459412 + 0.04576465*200 +  0.18853002*20 -0.00103749*20

In [ ]:
#El orden de (inv_TV, inv_Radio, inv_Newspaper) es importante: 
#en el mismo orden que al llamar a regr.fit(...)
regr.predict([[200,20,20]])

También podemos llamar a ``regr.predict()`` con un DataFrame como argumento, por ejemplo para predecir el nivel de ventas para distintas asignaciones alternativas de un presupuesto de 300k para inversión en publicidad.

**Atención**: es necesario que las columnas en el DataFrame aparezcan *en el mismo orden* que usamos al entrenar el modelo. Se debe a que ``scikit-learn`` fue concebida pensando en ``numpy``, antes de que ``pandas`` se hiciera tan popular.

In [ ]:
#No es necesario que tenga la respuesta
advertising_future = pd.DataFrame(
    [
        [100,30,30],
        [100,40,30],
        [100,30,40],
        [100,30,20]        
    ],
    columns=['TV', 'Radio', 'Newspaper']
)
regr.predict(advertising_future)

### Statsmodels 

Vamos a usar también la librería statsmodel, que imita la sintaxis de la regresión lineal en ``R``.

La fórmula ``Sales ~ TV + Radio`` significa que busca el modelo de regresión lineal:
$$
Sales\approx \beta_0 + \beta_1\times TV + \beta_2 \times Radio + \varepsilon
$$

 - Ajustamos el modelo $Sales\approx \beta_0 + \beta_1\times TV + \beta_2 \times Radio+ \varepsilon$ usando el DataFrame ``advertising``.
 
```python
recta = smf.ols('Sales ~ TV + Radio', advertising).fit()
```

In [ ]:
est = smf.ols('Sales ~ TV + Radio', advertising).fit()

### Usar el modelo para hacer predicciones

Usando **statsmodels**, necesitamos un DataFrame con las columnas predictoras, no es necesario que tenga la respuesta.

In [ ]:
advertising_future = pd.DataFrame(
    [
        [100,30,30],
        [100,40,30],
        [100,30,40],
        [100,30,20]        
    ],
    columns=['TV', 'Radio', 'Newspaper']
)
advertising_future

In [ ]:
est.predict(advertising_future)

In [ ]:
advertising_future = pd.DataFrame({
        'TV':[100,100,100,100],
        'Radio':[30,40,30,30],
        'Newspaper':[30,30,40,20]
})
advertising_future

In [ ]:
est.predict(advertising_future)

In [ ]:
advertising_future = pd.DataFrame(
    [
        [100,30,30],
        [100,40,30],
        [100,30,40],
        [100,30,20]        
    ],
    columns=['TV', 'Radio', 'Newspaper']
)

est.predict(advertising_future)

In [ ]:
df = pd.DataFrame({
    'TV': [100, 200, 250, 300],
    'Radio': [75, 50, 25, 0],
    'Newspaper': [75, 50, 25, 0]
})
est.predict(df)

### Interpretando el modelo ajustado

In [ ]:
est = smf.ols('Sales ~ TV + Radio + Newspaper', advertising).fit()
est.summary().tables[1]

In [ ]:
est = smf.ols('Sales ~ TV + Radio + Newspaper', advertising).fit()
est.summary()

### Intervalo de confianza

En la tabla resumen de statsmodel, podemos observar dos columnas ``[0.025 	0.975]``, que delimitan un **intervalo de confianza al 95%** para cada coeficiente.

No vamos a estudiar los intervalos de confianza en detalle, sólo diremos que:

 - Representan **un intervalo de _"valores razonables"_ para el valor de cada coeficiente** $\beta_0,\dots \beta_p$.
 - **No significa** que _"la probabilidad de que $\beta_j$ esté dentro del intervalo es 0.95"_.
 - No hay una distribución de probabilidad conjunta para los valores de los coeficientes. La definición, y la lógica detrás de los intervalos de confianza, es distinta.

### Contrastes de hipótesis

No vamos a estudiar los contrastes de hipótesis en detalle, sólo diremos que:

 - Lanzamos una hipótesis binaria como:
     - "Esta muestra es una extracción aleatoria independiente de una Normal con media 0 y desviación típica 20 cm".
     - "Estas dos muestras provienen de la misma distribución".
     - "El coeficiente de X en la regresión $Y\approx \beta_0 + \beta_1 X + \varepsilon$ es distinto de 0".
 - No asignamos una probabilidad a la hipótesis, sino que la respuesta es "sí (hay evidencia significativa de que la hipótesis es correcta)" o "no (no hay evidencia significativa de que la hipótesis es correcta)".
 
> - Es necesario poner mucha atención en distinguir _"**no** hay evidencia significativa de que la hipótesis es correcta"_ de _"hay evidencia significativa de que la hipótesis **no** es correcta"_.

 - Sin embargo, la confianza en la respuesta se mide con un **p-valor**, que **no es una probabilidad**.

### Contraste de hipótesis: "¿Podria este coeficiente ser cero?"

Observamos ahora las columnas ``std err 	t 	P>|t|``:
 - **std err** es una estimación del error estándar cometido en la estimación de cada coeficiente
 - **t**: el [estadístico t](https://en.wikipedia.org/wiki/T-statistic) es el ratio entre la estimación del coeficiente y el error estándar.
 - **P>|t|**: el **p-valor** asociado al contraste de hipótesis "¿Es posible que el coeficiente en realidad sea 0?".

   - La **hipótesis nula**: El valor del coeficiente $\beta_j$ es 0.
   - La **hipótesis alternativa**: El valor del coeficiente $\beta_j$ es **distinto de** 0.

> - Un valor bajo del p-valor indica que hay evidencia de que el coeficiente es distinto de cero.
> - Un valor alto del p-valor indica que no hay evidencia de que el coeficiente sea distinto de cero.

Si ajustamos el modelo lineal que depende sólo de Newspaper

$$
Sales\approx \beta_0 + \beta_3\times newspaper + \varepsilon,
$$

nos sale que la pendiente de la recta es positiva con un p-valor del 0.1%, pero si ajustamos el modelo lineal completo:

$$
Sales\approx \beta_0 + \beta_1\times TV  + \beta_2\times radio  + \beta_3\times newspaper + \varepsilon
$$

la pendiente $\beta_3$ es muy próxima a 0 (el p-valor es muy alto, el intervalo de confianza contiene a 0...)

In [ ]:
est = smf.ols('Sales ~ Radio', advertising).fit()
est.summary().tables[1]

In [ ]:
est = smf.ols('Sales ~ Newspaper', advertising).fit()
est.summary().tables[1]

En la tabla de resumen para regresión múltiple, aparece una fila para cada coeficiente del modelo lineal.

In [ ]:
est = smf.ols('Sales ~ TV + Radio + Newspaper', advertising).fit()
est.summary().tables[1]

¿Qué sentido tiene que el coeficiente de **newspaper** sea positivo en la regresión simple y cercano a 0 en la regresión múltiple? Se puede interpretar así:

> En aquellos países en los que se invierte más en anuncios en periódicos, también se suele invertir en TV y radio (lo comprobamos al observar que la correlación entre **newspaper** y **Radio** es del 35%). El modelo lineal múltiple indica que los anuncios en TV y radio son eficaces (aumentan las ventas), mientras que los anuncios en periódicos no. Sin embargo, si hacemos la regresión simple, resulta que los países en los que más se invierte más en anuncios en periódicos tienen más ventas que aquellos en los que se invierte menos, pero es debido a que también se invierte más en Radio, y no es causa directa de los anuncios en periódicos.

Un ejemplo clásico:
> Ajustamos un modelo lineal a $Y$ (ataques de tiburones) contra $X_1$ (ventas de helados en la playa) y encontramos una pendiente (correlación) positiva: a mayor ventas de helados, más ataques de tiburones. Obviamente no hay relación de causalidad entre ambas variables, pero en este caso podemos encontrar una causa que explica esta correlación: cuanta más gente en la playa ($X_2$), más ventas de helados y más ataques a tiburones. Una regresión múltiple $Y\sim X_1,X_2$, no muestra relación positiva entre $X_1$ e $Y$, y sí lo hace entre $X_2$ e $Y$.

Para más información, puedes leer sobre [factores de confusión](https://en.wikipedia.org/wiki/Confounding).

In [ ]:
advertising.corr()

## ¿Cómo decidimos si un modelo es bueno?

Supongamos que el mejor modelo lineal que hemos encontrado es 
$$
y=f(\mathbf{x})+\epsilon=\beta_0 + x_1\beta_1 +\dots +x_p\beta_p+\epsilon,\qquad \mathbf{x}=(x_1,\dots,x_p).
$$

 - **TSS**: Total sum of squares: $\Sigma_i (y_i-\bar{y})^2$ (sumamos el cuadrado de la diferencia entre el dato $y_i$ y la media $\bar{y}$)
 - **RSS**: Residual sum of squares:  $\Sigma_i (y_i-f(\mathbf{x}_i))^2$ (sumamos el cuadrado de la diferencia entre el dato $y_i$ y la predicción $f(\mathbf{x}_i)$ usando nuestro modelo)
 - **RSS/TSS**: cociente entre la "varianza residual" y la "varianza total"
 - **R-cuadrado**: "porcentaje de la varianza que el modelo explica". 
$$
R^2 = 1 - \frac{RSS}{TSS}
$$

```python
#Calcula R^2 con scikit-learn
regr.score(X,y)
```

```python
#Devuelve R^2 para el modelo ajustado con statsmodels
est.rsquared
```

In [ ]:
#Calcula R^2 con scikit-learn
regr.score(X,y)

In [ ]:
#Devuelve R^2 para el modelo ajustado con statsmodels
est.rsquared

El estadístico $R^2$ es igual al cuadrado de la correlación entre la respuesta y el valor del modelo lineal ajustado.
$$
R^2 = Cor(Y,f(X))^2
$$
Sin embargo, es necesario ajustar el modelo para calcular esa correlación, no es una correlación entre las variables originales, como ocurría en el caso de la regresión lineal simple.

## Comparando modelos

El estadístico $R^2$ siempre aumenta al añadir variables, pero ¿añadir más y más variables da lugar siempre a un modelo mejor?

A menudo, *la ganancia marginal por incorporar otra variable no compensa las desventajas*. Un modelo con más variables, o con términos no lineales:

 1. es más complejo, y nos impide usar el modelo de ciertas formas.
 2. **generaliza** peor: aunque ajuste muy bien a los datos que tenemos, ajusta peor a datos nuevos.

La semana que viene hablaremos despacio sobre los temas de **sobreajuste**, **validación** y **sesgo vs varianza**.
Por el momento, sólo mencionaremos que hay medidores de bondad de ajuste alternativos al **R-squared** que intentan buscar un equilibrio entre un buen ajuste y un modelo sencillo con pocas variables predictoras:

 - **Adj. R-squared** aumenta cuando decrece el error, pero disminuye cuando incluimos más variables predictoras. De hecho, la fórmula es sencilla:
 
$$
{\bar {R}}^{2}={1-(1-R^{2}){n-1 \over n-p-1}}
$$

 - **AIC** y **BIC** disminuyen cuando decrece el error, pero aumentan cuando incluimos más variables predictoras. Se definen de forma similar, pero el BIC penaliza más duramente los modelos con más parámetros.
 
> _Tanto el AIC como el BIC solo sirven para comparar modelos. No importa si son grandes o pequeños._
> 
> Si el AIC para el modelo ``A`` vale a y el AIC para el modelo ``B`` vale b, entonces es $\exp\left(\frac{a-b}{2}\right)$ veces más probable que el modelo A minimize la pérdida de información que el modelo B (si son igual de probables a priori). Este ratio es exactamente el mismo si $a=2, b=1$ o si $a=3001, b=3000$.

### Comparando modelos en el ejemplo de prensa

Tras observar que el coeficiente de **newspaper** tiene un p-valor alto (o que 0 está en un intervalo de confianza), probamos a usar el modelo que sólo usa TV y Radio.

Comparamos los resultados de ambos modelos.

In [ ]:
est = smf.ols('Sales ~ TV + Radio + Newspaper', advertising).fit()
est.summary().tables[0]

In [ ]:
est = smf.ols('Sales ~ TV + Radio', advertising).fit()
est.summary().tables[0]

Vemos que sin la variable **newspaper** obtenemos casi el mismo valor de R-squared (explicamos el mismo porcentaje de la varianza). Los indicadores AIC y BIC disminuyen ligeramente, confirmando que es buen movimiento. El _"adjusted $R^2$"_ es casi igual en ambos casos: es un criterio que no es muy crítico con el aumento del número de variables predictoras. De los tres criterios, el BIC es el que más penaliza el aumento de variables redundantes.

## Variables categóricas

> ¿Podemos incorporar variables cualitativas a una regresión?

En el dataset siguiente, se trata de predecir el balance de una tarjeta de crédito en función de varias variables:

In [ ]:
credit = pd.read_csv('credit.csv')
credit.head()

Por ejemplo, podemos intentar decidir si el género es una buena variable predictora, ajustando un modelo:

> Balance ~ Gender

que es lo mismo que ajustar

$$
\text{Balance} = \beta_0 + \beta_1 x_{\text{Gender}}  + \varepsilon
$$

donde $x_{\text{Gender}}$ vale 1 si es "Male" y 0 si es "Female".

In [ ]:
est = smf.ols('Balance ~ Gender', credit).fit()
print(est.rsquared)
est.summary().tables[1]

La conclusión parece ser que no hay evidencia a favor de una relación entre Género y balance.

También podemos incorporar como variable predictora una variable categórica que toma más de dos valores, como por ejemplo ``Ethnicity``:

> Balance ~ Ethnicity

pero cuidado, **no se trata de codificar** cada valor de la variable categórica con una valor entero distinto:

- African American => 1
- Caucasian => 2
- Asian => 3

y luego ajustar un modelo

$$
\text{Balance} = \beta_0 + \beta_1 x_{\text{Ethnicity}} + \varepsilon
$$

porque estaríamos imponiendo que el efecto de "Asian" es el triple que el de "African American", en vez de dejar que lo elija el modelo.
Lo que hacemos es usar varias variables binarias:

 - $x_{\text{Caucasian}}$: vale 1 si es "Caucasian", y 0 en cualquiera de los otros dos casos.
 - $x_{\text{Asian}}$: vale 1 si es "Asian", y 0 en cualquiera de los otros dos casos.
 
El modelo es:

$$
\text{Balance} = \beta_0 + \beta_1 x_{\text{Asian}} + \beta_2 x_{\text{Caucasian}} + \varepsilon
$$
 
**Pregunta**: ¿por qué no necesitamos otra variable $x_{\text{African American}}$?

In [ ]:
est = smf.ols('Balance ~ Ethnicity', credit).fit()
print(est.rsquared)
est.summary().tables[1]

In [ ]:
est = smf.ols('Balance ~ Income + Ethnicity', credit).fit()
print(est.rsquared)
est.summary().tables[1]

### Ejercicio 1

Trabajaremos con la tabla `tips.csv`:
- Ajusta un modelo lineal a los datos para predecir la propina (tip) en función del resto de columnas: ¿obtienes buenos resultados?
- Encuentra el modelo de regresión lineal múltiple que tenga mejor BIC.
- ¿Crees que el sexo de la persona que deja la propina es indicativa de si será más o menos generosa?

Comenzamos incluyendo todas las variables

In [ ]:
tips = pd.read_csv('tips.csv')
tips.describe(include='all')

### Ejercicio 2

 - Carga el conjunto ``seaice.csv``, sobre la extensión del hielo polar.
 - Selecciona sólo los datos de extensión del hielo en el hemisferio norte.
 - Intenta explicar los datos de extensión del hielo polar en función del año.
 - Intenta explicar los datos de extensión del hielo polar en función del año y del mes (pero *atención*: ¿cómo tiene sentido incorporar el mes?). ¿El modelo es mejor que el que usa sólo los datos de año?
 - Usa el modelo para extrapolar la extensión del hielo en el Oceáno Ártico en Julio de 2030, 2040 y 2050.
 


**Nota**: puedes usar `... + C(nombre_variable) + ...` para convertir una variable entera en categórica. Es decir, si ajustamos un modelo

>    Extensión del hielo ~ Mes

Estamos imponiendo que el efecto del mes de Diciembre es 12 veces el efecto del mes de Enero.

Sin embargo, si usamos

>    Extensión del hielo ~ C(Mes)

Estamos ajustando un modelo

$$
    E = \beta_0 + \beta_1 x_{ENERO} + \dots + \beta_{12} x_{DICIEMBRE}  + \varepsilon
$$

donde la contribución del mes de Diciembre es independiente de la contribución del efecto del mes de Enero.
Observa el ejemplo debajo:

In [ ]:
ice = pd.read_csv('seaice.csv')
ice.info()

In [ ]:
icen = ice[ice.hemisphere=='north']
est = smf.ols('Extent ~ Month', icen).fit()
est.summary()

In [ ]:
icen = ice[ice.hemisphere=='north']
est = smf.ols('Extent ~ C(Month)', icen).fit()
est.summary()

### Ejercicio 3

 - Carga el conjunto ``simar.csv``, sobre los datos de viento y oleaje de una boya del puerto de Cádiz.
 - Intenta explicar los datos de altura significativa de ola (Hm0) en función del mes. (pero *atención*: ¿cómo tiene sentido incorporar el mes?).
 - Intenta explicar los datos de altura significativa de ola (Hm0) en función del mes y de la hora.
 - Intenta explicar los datos de altura significativa de ola (Hm0) en función del mes, la hora y el año.
 - ¿Has conseguido un buen modelo?

In [ ]:
simar=pd.read_csv('SIMAR_fake.csv')
simar.head()